现在的问题是可以做的办法太麻烦

不好做的部分又不可行

In [1]:
3000 * 2500

7500000

In [21]:
import wrds
db = wrds.Connection(wrds_username='kpu2019')

Loading library list...
Done


In [28]:
db.

['aha_sample',
 'ahasamp',
 'audit',
 'audit_audit_comp',
 'audit_corp_legal',
 'block',
 'boardex_trial',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'centris',
 'ciq',
 'ciq_common',
 'clrvt',
 'clrvtsmp',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global',
 'comp_global_daily',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'compa',
 'compb',
 'compbd',
 'compd',
 'compdcur',
 'compg',
 'compgd',
 'compm',
 'compnad',
 'compsamp_snapshot',
 'compseg',
 'compsegd',
 'contrib',
 'contrib_char_returns',
 'crsp',
 'crsp_a_ccm',
 'crsp_a_indexes',
 'crsp_a_stock',
 'crsp_a_treasuries',
 'crsp_q_indexhist',
 'crsp_q_mutualfunds',
 'crspa',
 'crspm',
 'crspq',
 'csmar',
 'csmar_cg',
 'dealscan',
 'djones',
 'dmef',
 'doe',
 'etfg_samp',
 'etfgsamp',
 'eureka',
 'eurekahedge_sample',
 'execcomp',
 'factsamp_revere',
 'ff',
 'ff_all_new',
 'frb',
 'frb_all_new',
 'ftse',
 'ginsight',
 'gmi',
 'gmi_all_new',
 'gsi',
 'hfr',
 'hfrsamp_hfrdb',
 'ibes',
 'ibeskpi',

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import scipy
# import tushare as ts
import wrds
import talib as ta
import datetime, time


In [4]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

retrieve data from csv

In [16]:
data_fund = pd.read_csv('data/query7-replace some data in query6.zip',
                         parse_dates=['datadate'], infer_datetime_format=True,
                         dtype={'cusip':str, 'gvkey':str})

In [6]:
data_return = pd.read_csv('data/return.zip', parse_dates=['date'], infer_datetime_format=True,
                  dtype={'PERMCO':str, 'CUSIP':str, 'PERMNO':str})
# data = pd.read_csv('query3.csv', parse_dates=['date'], infer_datetime_format=True,)
                   #dtype={'PRC':np.float64, 'SHROUT':np.float64})
data_return = data_return.set_index('date')
data_return.RET = data_return.RET.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)
data_return.RETX = data_return.RETX.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)
# data_return = data_return.assign(MV=data_return.PRC.abs() * data_return.SHROUT,
#                           RET_PLUS=data_return.RET+1,
#                           RETX_PLUS=data_return.RETX+1)
# data_return = data_return.set_index(['date', 'PERMCO'])

看一下两个表各有多少个标的

In [29]:
data_fund.head()

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,conm,curcdq,datacqtr,datafqtr,dlttq,oiadpq,pstkq,exchg,costat,mkvaltq
0,001000,1973-03-31,1973,1,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,USD,1973Q1,1973Q1,7.0,NaN,0.0,12.0,I,NaN
1,001000,1973-06-30,1973,2,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,USD,1973Q2,1973Q2,7.0,NaN,0.0,12.0,I,NaN
2,001000,1973-09-30,1973,3,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,USD,1973Q3,1973Q3,7.0,NaN,0.0,12.0,I,NaN
3,001000,1973-12-31,1973,4,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,USD,1973Q4,1973Q4,7.0,3.837,0.0,12.0,I,NaN
4,001000,1974-03-31,1974,1,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,USD,1974Q1,1974Q1,7.0,NaN,0.0,12.0,I,NaN


In [17]:
data_fund.gvkey.value_counts()

010787    147
009538    146
003969    146
002538    146
001327    146
009611    146
003708    146
011669    146
006534    146
008387    146
007750    146
002061    146
002497    146
008240    146
002791    146
011537    146
008633    145
006506    145
002916    145
003639    145
008219    145
011220    145
002271    145
010862    145
011296    145
007733    145
010801    145
006268    145
007152    145
005639    145
         ... 
165230      1
177942      1
181875      1
181876      1
178207      1
174365      1
009286      1
142603      1
180506      1
181733      1
181651      1
181873      1
181701      1
177565      1
010074      1
001175      1
181653      1
181654      1
183731      1
181272      1
178209      1
142501      1
184689      1
144103      1
181729      1
181912      1
178303      1
181728      1
186725      1
181703      1
Name: gvkey, Length: 29489, dtype: int64

In [32]:
data_fund.cusip.value_counts().index

Index(['G91442106', '95058W100', '817565104', '502470107', '254543101',
       '988858106', '068798107', '811543107', '576323109', '88033G407',
       ...
       '652507104', '20824F108', '008086100', '71646R309', '25459Y694',
       '46429K101', '760113100', '464288166', '74623K107', '74347W353'],
      dtype='object', length=29476)

In [14]:
data_return.shape

(2986697, 9)

In [12]:
data_return.CUSIP.value_counts()

03976210    432
14912310    432
12770310    432
29605610    432
16808810    432
25454310    432
17123210    432
05534B76    432
40423210    432
84790A10    432
82654610    432
85206110    432
43147510    432
78463510    432
26414710    432
50069210    432
60855410    432
55277P10    432
04418610    432
G5150210    432
63754610    432
90467720    432
45920010    432
46981410    432
87655330    432
35351410    432
15670010    432
29101110    432
29482160    432
96214910    432
           ... 
60872720      1
37038810      1
85356410      1
25490K81      1
97239110      1
58133410      1
25459W73      1
32529710      1
78464A33      1
92828Q10      1
92211910      1
69347810      1
19650410      1
00923010      1
13680710      1
55171510      1
46428811      1
25720110      1
29642510      1
29383920      1
67984210      1
83172F20      1
92189F83      1
97209910      1
37989210      1
23323710      1
90253910      1
68386C60      1
59425310      1
89099090      1
Name: CUSIP, Length: 265

In [37]:
len(set(data_fund.cusip.value_counts().index))

29476

In [36]:
len(set(data_fund.cusip.value_counts().index) - set(data_return.CUSIP.value_counts()))

29476

In [18]:
data_return.PERMCO.value_counts()

37493    7953
31600    3287
44072    2615
34961    2284
39147    1764
50846    1494
50852    1210
34957    1098
21438     864
20340     864
20945     864
20799     791
21219     776
21422     770
655       760
4922      751
20751     735
925       728
21893     728
2519      726
44040     721
4428      712
41593     710
714       705
1015      702
2881      690
4388      682
21597     674
3548      663
21625     660
         ... 
23371       2
24925       1
25248       1
25200       1
23768       1
25624       1
25972       1
24743       1
24706       1
23987       1
25948       1
53114       1
24126       1
23977       1
24227       1
129         1
23281       1
24334       1
24342       1
23983       1
53113       1
53116       1
25604       1
25124       1
25365       1
23692       1
24464       1
25647       1
25250       1
53115       1
Name: PERMCO, Length: 25097, dtype: int64

In [30]:
data_return.PERMNO.value_counts()

39220    432
12060    432
59459    432
47888    432
59619    432
57330    432
13856    432
29209    432
40416    432
80857    432
44513    432
14277    432
12052    432
26657    432
48523    432
38156    432
61209    432
14868    432
60097    432
33188    432
58683    432
17478    432
59328    432
21726    432
36397    432
14795    432
52476    432
42796    432
11674    432
49488    432
        ... 
47212      1
60469      1
92871      1
52409      1
59353      1
52417      1
58326      1
24598      1
40491      1
92886      1
43756      1
50316      1
34120      1
36055      1
80291      1
54316      1
34243      1
92880      1
92876      1
92881      1
92885      1
34200      1
15094      1
92884      1
51705      1
92883      1
92875      1
44249      1
92878      1
77738      1
Name: PERMNO, Length: 26501, dtype: int64

In [5]:
def calc_basis(dataframe):
    PRC = dataframe.PRC
    SHROUT = dataframe.SHROUT
    CUSIP = dataframe.CUSIP
    MV = PRC.abs() * SHROUT
    RET_CUM = (dataframe.RET + 1).cumprod()
    RETX_CUM = (dataframe.RETX + 1).cumprod()
    
    return pd.DataFrame(
        {
            'CUSIP':CUSIP,
            'PRC':PRC,
            'SHROUT':SHROUT,
            'MV':MV,
            'RET_CUM':RET_CUM,
            'RETX_CUM':RETX_CUM
        }
    )

In [ ]:
data_ret_cum = data_return.groupby(['PERMCO', pd.Grouper(freq="Y")]).\
apply(lambda x : calc_basis(x))

In [25]:
data_return.head(1)

,,PERMNO,TICKER,COMNAM,CUSIP,PRC,RET,SHROUT,RETX,MV,RET_PLUS,RETX_PLUS
date,PERMCO,,,,,,,,,,,
1973-01-31,2,14403,AAGIY,ANGLO AMERICAN GOLD INVT LTD,03487050,-27.125,0.179348,21952.0,0.179348,595448.0,1.179348,1.179348


In [ ]:
data_return_year = data_return.groupby(level=1, sort=False).resample('Y', level=0).apply(
                    {
                        'PERMNO':'last', 
                        'TICKER':'last', 
                        'COMNAM':'last', 
                        'CUSIP':'last', 
                        'PRC':'last', 
                        'RET':'last', 
                        'SHROUT':'last', 
                        'RETX':'last',
                        'MV':'last', 
                        'RET_PLUS':'cumprod', 
                        'RETX_PLUS':'cumprod'
                    })

In [1]:
# data_query7.head()

calc MV & CUM RETURNX

In [5]:
cum_ret = data.groupby(['PERMCO', pd.Grouper(freq="Y")]).apply(lambda x : x.RETPLUS.cumprod())
cum_ret.index = cum_ret.index.droplevel(1)
cum_ret = cum_ret.reset_index()
cum_ret.columns = ['PERMCO', 'date', 'CUMRETX']
cum_ret = cum_ret.set_index('date')
cum_ret.head()

,PERMCO,date,CUMRETX
0,100,1973-01-31,0.763819
1,100,1973-02-28,0.733668
2,100,1973-03-30,0.713568
3,100,1973-04-30,0.693467
4,100,1973-05-31,0.542714
